In [ ]:
pip install --upgrade transformers


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
from datasets import load_dataset
dataset = load_dataset("iSemantics/conllpp-ner-ar")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/843 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/738k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/184k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/167k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2383 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2572 [00:00<?, ? examples/s]

In [ ]:
import re

def clean_data(data):
  data =re.sub(r'[^\w\s]','',data)
  data =re.sub(r'\s+','',data).strip()
  return data

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")



tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def clean_invalid_tags(dataset):
    # Define label_list here, before it's used in the filter function
    label_list = dataset["train"].features["ner_tags"].feature.names

    for split in  ['train', 'validation', 'test']:  # تحقق في كل تقسيم
        print(f"Cleaning {split} dataset...")
        # تصفية البيانات
        dataset[split] = dataset[split].filter(lambda example: all(tag < len(label_list) for tag in example["ner_tags"]))
        # طباعة عدد الأمثلة بعد التصفية
        print(f"Number of examples in {split} after cleaning: {len(dataset[split])}")

# تنظيف البيانات
clean_invalid_tags(dataset)

Cleaning train dataset...


Filter:   0%|          | 0/10250 [00:00<?, ? examples/s]

Number of examples in train after cleaning: 10250
Cleaning validation dataset...


Filter:   0%|          | 0/2383 [00:00<?, ? examples/s]

Number of examples in validation after cleaning: 2383
Cleaning test dataset...


Filter:   0%|          | 0/2572 [00:00<?, ? examples/s]

Number of examples in test after cleaning: 2572


entity labeling using (BIO Scheme)

In [ ]:
label_list =dataset["train"].features["ner_tags"].feature.names
#label2id ={label: i for i,label in enumerate(label_list)} ما نحتاجها هنا

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], padding="max_length", truncation=True, max_length=128, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label[word_idx] != -100 else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
tokenized_train = dataset["train"].map(tokenize_and_align_labels, batched=True)
tokenized_val = dataset["validation"].map(tokenize_and_align_labels, batched=True)
tokenized_test = dataset["test"].map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/10250 [00:00<?, ? examples/s]

Map:   0%|          | 0/2383 [00:00<?, ? examples/s]

Map:   0%|          | 0/2572 [00:00<?, ? examples/s]

Label Mapping


In [ ]:
id2label = {idx: label for idx, label in enumerate(label_list)}
label2id = {label: idx for idx, label in enumerate(label_list)}

Import and load the Pre-trained Model

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "aubmindlab/bert-base-arabertv02",  # Pre-trained AraBERT model
    num_labels=len(label_list),        # Number of unique NER tags
    id2label=id2label,                 # Map from ID to label
    label2id=label2id                  # Map from label to ID
)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Traning

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# Import necessary tools from Hugging Face Transformers and scikit-learn
from transformers import Trainer, TrainingArguments, AutoModelForTokenClassification, DataCollatorForTokenClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

# Create a data collator for token classification (NER task)
# This ensures input batches are padded and formatted correctly
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define a function to compute evaluation metrics (accuracy, precision, recall, F1)
def compute_metrics(p):
    predictions, labels = p
    # Select the most likely class for each token
    predictions = np.argmax(predictions, axis=2)

    # Remove padding tokens (with label -100) from evaluation
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Compute evaluation metrics using flattened label lists
    precision, recall, f1, _ = precision_recall_fscore_support(
        sum(true_labels, []),
        sum(true_predictions, []),
        average="macro"  # Calculate macro-average across all classes
    )
    acc = accuracy_score(sum(true_labels, []), sum(true_predictions, []))  # Compute overall accuracy
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# Define the list of possible labels for NER
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


# Set up training arguments (number of epochs, batch size, logging, etc.)
training_args = TrainingArguments(
    output_dir="./results",              # Where to save model checkpoints and outputs
    per_device_train_batch_size=16,      # Training batch size
    per_device_eval_batch_size=16,       # Evaluation batch size
    num_train_epochs=5,                  # Total number of training epochs
    weight_decay=0.01,                   # Weight decay to avoid overfitting
    logging_dir="./logs",                # Directory for training logs
    logging_steps=10,                    # Log every 10 steps
    save_steps=500,                      # Save model every 500 steps
    do_eval=True,                        # Run evaluation during training
    do_train=True,                       # Enable training
    fp16=True,                           # Enable mixed-precision training (16-bit floats)
)

# Initialize the Trainer class that handles training and evaluation
trainer = Trainer(
    model=model,                         # The model to train
    args=training_args,                  # Training arguments
    train_dataset=tokenized_train,       # Preprocessed training dataset
    eval_dataset=tokenized_val,          # Preprocessed validation dataset
    tokenizer=tokenizer,                 # Tokenizer used for preprocessing
    data_collator=data_collator,         # Data collator for token classification
    compute_metrics=compute_metrics,     # Evaluation function
)

# Start the training process
trainer.train()


<ipython-input-13-fdc91560fb62>:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 444008978 (444008978-princess-nourah) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.417300
20,0.681000
30,0.540000
40,0.523300
50,0.354600
60,0.440800
70,0.382400
80,0.357300
90,0.328900
100,0.325600


TrainOutput(global_step=3205, training_loss=0.11028702687966098, metrics={'train_runtime': 740.129, 'train_samples_per_second': 69.245, 'train_steps_per_second': 4.33, 'total_flos': 3348076570560000.0, 'train_loss': 0.11028702687966098, 'epoch': 5.0})

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Evaluate the model on the validation set
eval_results = trainer.evaluate(tokenized_val)

# Print the basic evaluation metrics
print("\n" + "="*50)
print("Final Evaluation Results on Validation Set:")
print("="*50)
print(f"{'Loss:':<20} {eval_results['eval_loss']:.4f}")
print(f"{'Accuracy:':<20} {eval_results['eval_accuracy']:.4f}")
print(f"{'F1 Score:':<20} {eval_results['eval_f1']:.4f}")
print(f"{'Precision:':<20} {eval_results['eval_precision']:.4f}")
print(f"{'Recall:':<20} {eval_results['eval_recall']:.4f}")

# Function to generate a classification report
def get_classification_report(predictions, labels):
    # Remove padding tokens (where label = -100)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists
    flat_predictions = [item for sublist in true_predictions for item in sublist]
    flat_labels = [item for sublist in true_labels for item in sublist]

    # Generate the classification report
    report = classification_report(flat_labels, flat_predictions, output_dict=True)
    return report, flat_labels, flat_predictions

# Get predictions from the model
predictions, labels, _ = trainer.predict(tokenized_val)
predictions = np.argmax(predictions, axis=2)

# Get the detailed report
detailed_report, true_labels, pred_labels = get_classification_report(predictions, labels)

# Print the detailed classification report
print("\n" + "="*50)
print("Detailed Classification Report:")
print("="*50)
print(classification_report(true_labels, pred_labels))

# Print performance analysis per class
print("\n" + "="*50)
print("Per-Class Performance Analysis:")
print("="*50)
for label in label_list:
    if label in detailed_report:  # Skip averages
        print(f"\nPerformance for class {label}:")
        print(f"- Precision: {detailed_report[label]['precision']:.3f}")
        print(f"- Recall: {detailed_report[label]['recall']:.3f}")
        print(f"- F1 Score: {detailed_report[label]['f1-score']:.3f}")
        print(f"- Support: {detailed_report[label]['support']}")



Final Evaluation Results on Validation Set:
Loss:                0.2112
Accuracy:            0.9550
F1 Score:            0.8523
Precision:           0.8742
Recall:              0.8360

Detailed Classification Report:
              precision    recall  f1-score   support

       B-LOC       0.86      0.88      0.87      1646
      B-MISC       0.77      0.71      0.73       684
       B-ORG       0.91      0.90      0.90      3013
       B-PER       0.91      0.95      0.93      2585
       I-LOC       0.78      0.76      0.77       233
      I-MISC       0.84      0.59      0.69       357
       I-ORG       0.87      0.81      0.84      1574
       I-PER       0.95      0.95      0.95      2381
           O       0.98      0.98      0.98     31273

    accuracy                           0.96     43746
   macro avg       0.87      0.84      0.85     43746
weighted avg       0.95      0.96      0.95     43746


Per-Class Performance Analysis:

Performance for class O:
- Precision: 0.978

# **Testing and Save The Model**

In [ ]:
# Evaluate the model on the test set
test_results = trainer.evaluate(tokenized_test)

# Print evaluation metrics on the test set
print("\n" + "="*50)
print("Final Evaluation Results on Test Set:")
print("="*50)
print(f"{'Loss:':<20} {test_results['eval_loss']:.4f}")
print(f"{'Accuracy:':<20} {test_results['eval_accuracy']:.4f}")
print(f"{'F1 Score:':<20} {test_results['eval_f1']:.4f}")
print(f"{'Precision:':<20} {test_results['eval_precision']:.4f}")
print(f"{'Recall:':<20} {test_results['eval_recall']:.4f}")

# Get predictions on the test set
test_predictions, test_labels, _ = trainer.predict(tokenized_test)
test_predictions = np.argmax(test_predictions, axis=2)

# Generate detailed report
test_report, true_test_labels, pred_test_labels = get_classification_report(test_predictions, test_labels)

# Print the detailed classification report
print("\n" + "="*50)
print("Detailed Classification Report (Test Set):")
print("="*50)
print(classification_report(true_test_labels, pred_test_labels))

# Per-Class Performance Analysis
print("\n" + "="*50)
print("Per-Class Performance Analysis (Test Set):")
print("="*50)
for label in label_list:
    if label in test_report:
        print(f"\nPerformance for class {label}:")
        print(f"- Precision: {test_report[label]['precision']:.3f}")
        print(f"- Recall: {test_report[label]['recall']:.3f}")
        print(f"- F1 Score: {test_report[label]['f1-score']:.3f}")
        print(f"- Support: {test_report[label]['support']}")



Final Evaluation Results on Test Set:
Loss:                0.2889
Accuracy:            0.9442
F1 Score:            0.8256
Precision:           0.8251
Recall:              0.8264

Detailed Classification Report (Test Set):
              precision    recall  f1-score   support

       B-LOC       0.85      0.87      0.86      1722
      B-MISC       0.68      0.69      0.68       601
       B-ORG       0.88      0.87      0.87      3231
       B-PER       0.91      0.94      0.93      2317
       I-LOC       0.75      0.77      0.76       321
      I-MISC       0.61      0.57      0.59       284
       I-ORG       0.83      0.81      0.82      1836
       I-PER       0.94      0.95      0.95      2093
           O       0.98      0.98      0.98     28447

    accuracy                           0.94     40852
   macro avg       0.83      0.83      0.83     40852
weighted avg       0.94      0.94      0.94     40852


Per-Class Performance Analysis (Test Set):

Performance for class O:
- 

In [ ]:
# Save the trained model and tokenizer
model_path = "./saved_model_arabert_ner"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print(f"\nModel and tokenizer saved to {model_path}")



Model and tokenizer saved to ./saved_model_arabert_ner


# **Deployment**

In [ ]:
# Import the necessary libraries
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# Load the model and tokenizer from the local directory
model_path="saved_model_arabert_ner"
model = AutoModelForTokenClassification.from_pretrained(model_path) # Corrected path to the saved model
tokenizer = AutoTokenizer.from_pretrained(model_path) # Corrected path to the saved model


In [ ]:

# Create the NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


# Test sentence
sentence = "أعلن الملك سلمان بن عبدالعزيز عن دعم المملكة العربية السعودية لاتفاقية السلام في الشرق الأوسط خلال القمة العربية في القاهرة."

# Run prediction
ner_results = ner_pipeline(sentence)

# Display results
print("\n" + "="*50)
print("Named Entities Extracted:")
print("="*50)
for entity in ner_results:
    print(f"Entity: {entity['word']:<15} | Label: {entity['entity_group']:<10} | Score: {entity['score']:.2f}")


Device set to use cuda:0



Named Entities Extracted:
Entity: سلمان بن عبدالعزيز | Label: PER        | Score: 0.97
Entity: الشرق الأوسط    | Label: LOC        | Score: 0.98
Entity: العربية         | Label: MISC       | Score: 0.77
Entity: القاهرة         | Label: LOC        | Score: 0.97


Model Interface

In [ ]:

!ngrok config add-authtoken 


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:

!pip install streamlit pyngrok transformers -q

#save the streamlit code into file
code = """
import streamlit as st
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
from collections import Counter

model_path = "saved_model_arabert_ner"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

st.set_page_config(page_title="تحليل النص السياسي", layout="wide")

#header layout
st.markdown(
    '''
    <div style="
        background-image: url('https://th.bing.com/th/id/OIP.f0vgoGmlHu55CfsA3yacBgHaE7?pid=ImgDet&w=474&h=315&rs=1');
        background-size: 100% auto;
        background-position: center;
        image-rendering: -webkit-optimize-contrast;
        padding: 70px 20px;
        border-radius: 12px;
        text-align: center;
        border: 2px solid #006c35;
        box-shadow: 0 4px 10px rgba(0,0,0,0.15);
    ">
        <h2 style="color: white; font-size: 32px; text-shadow: 1px 1px 3px #000;">
             نظام استخراج الكيانات من النصوص السياسية
        </h2>
        <h4 style="color: white; font-size: 18px; margin-top: 15px; text-shadow: 1px 1px 3px #000;">
            دعمًا لرؤية 2030، يساهم هذا المشروع في التحول الرقمي وتحليل المحتوى السياسي لخدمة المملكة
        </h4>
    </div>
    <br>
    ''',
    unsafe_allow_html=True
)

#ready to test example
example_text = "أعلن الملك سلمان بن عبدالعزيز عن دعم المملكة العربية السعودية لاتفاقية السلام في الشرق الأوسط خلال القمة العربية في القاهرة."
use_example = st.button("🧪 تحليل نص تجريبي")

if use_example:
    st.markdown("**🔸 النص المستخدم:**")
    st.info(example_text)
    text_input = example_text
else:
    text_input = st.text_area("🌴 أدخل النص السياسي هنا:", height=300)

# فلترة حسب نوع الكيان
entity_options = ["كل الأنواع", "PER - أشخاص", "ORG - منظمات", "LOC - أماكن", "MISC - متنوع"]
selected_filter = st.selectbox("🔸 فلترة النتائج حسب نوع الكيان:", entity_options)

if st.button("🔍 تحليل النص") or use_example:
    if not text_input.strip():
        st.warning("رجاءً أدخل نصاً لتحليله.")
    else:
        results = ner_pipeline(text_input)

        if not results:
            st.info("لم يتم العثور على كيانات.")
        else:
            label_counter = Counter()
            filtered = []

            for entity in results:
                label_counter[entity["entity_group"]] += 1
                if selected_filter == "كل الأنواع" or selected_filter.startswith(entity["entity_group"]):
                    filtered.append(entity)

            st.markdown("<h4>🔸 الكيانات المستخرجة:</h4>", unsafe_allow_html=True)

            if filtered:
                for entity in filtered:
                    st.markdown(f'''
                        <div style="padding:10px; margin:10px 0; border:2px solid #006c35; border-radius:8px; background:#eaf7f0">
                            <b>الكلمة:</b> {entity['word']}<br>
                            <b>النوع:</b> {entity['entity_group']}<br>
                            <b>الثقة:</b> {round(entity['score'], 2)}
                        </div>
                    ''', unsafe_allow_html=True)
            else:
                st.warning("🚫 لا توجد كيانات من هذا النوع في النص.")

            # Insight Box: smart analysis of text
            most_common = label_counter.most_common(1)
            top_entity = most_common[0][0] if most_common else None

            insight = ""
            if top_entity == "PER":
                insight = " يركز النص على الشخصيات السياسية، مما يدل على اهتمام بالأفراد المؤثرين في المشهد السياسي."
            elif top_entity == "ORG":
                insight = " النص يتناول منظمات ومؤسسات، مما يشير إلى تحليل للمشهد السياسي."
            elif top_entity == "LOC":
                insight = " يحتوي النص على مواقع وأماكن، ما يدل على بعد جغرافي في الطرح السياسي."
            elif top_entity == "MISC":
                insight = " النص يشير إلى كيانات متنوعة، ما يدل على طيف واسع من المواضيع المطروحة."

            if insight:
                st.markdown("<h4> تحليل ذكي للنص السياسي بالكامل:</h4>", unsafe_allow_html=True)
                st.success(insight)
"""



with open("ner_app.py", "w", encoding="utf-8") as f:
    f.write(code)

# Start Streamlit
from pyngrok import ngrok
import time

ngrok.kill() #termenate any other ngrok session
!streamlit run ner_app.py &>/content/log.txt &


time.sleep(5)


public_url = ngrok.connect(addr="8501")
print(f"\n👉 {public_url}")



👉 NgrokTunnel: "https://914c-35-185-18-147.ngrok-free.app" -> "http://localhost:8501"
